In [1]:
import sys
sys.path.insert(0, '..')
# Useful starting lines
%matplotlib inline
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2

In [2]:
import Helpers.helpers as helper
import Helpers.cleaner as cleaner
import numpy as np
import Helpers.implementations as imp

In [3]:
part = [0,1,2,3]
data = []
for p in part:
    data.append(helper.load_clean_data(f'../../data/clean/x_train_{p}.npy', f'../../data/clean/y_train_{p}.npy'))

In [4]:
print(data[0][0].shape)
print(data[1][0].shape)
print(data[2][0].shape)
print(data[3][0].shape)

(99913, 17)
(77544, 21)
(50379, 29)
(22164, 29)


In [5]:
def toDeg(x,degree):
    res = x
    for i in range(2,degree +1):
        power = np.power(x,i)
        res = np.c_[res,power]
    return res

def addConstant(x):
    return np.c_[np.ones((x.shape[0],1)),x]

In [5]:
x = [data[p][0] for p in part]
y = [data[p][1] for p in part]

xpow = [helper.addConstant(helper.toDeg(x[p], 2)) for p in part]

In [49]:
def train_model_ridge_cross(x, y, lambdas=[-0.00001], ratio=.8, seed=1):
    accuracies = []
    ws = []
    rmse_trs = []
    rmse_tes = []
    
    x_tr,y_tr,x_te,y_te = imp.split_data(x,y,ratio,seed)
    w_all, rmse_tr, rmse_te = imp.full_cross_validation(x_tr,y_tr, lambdas=lambdas)
    
    for w in w_all:
        y_pred = helper.predict_labels(w,x_te)
        res = np.array([(1 if(y_pred[i] == y_te[i]) else 0) for i in range(y_te.shape[0])])
        accuracies.append(res.sum()/len(res))
        
        
    i = np.argmax(accuracies)
    return w_all[i], accuracies[i], rmse_tr[i], rmse_te[i]

In [55]:
def train_all_models(xs, ys, lambdas=[0.001, .01, -0.01], parts=[0,1,2,3]):
    xpow = [helper.addConstant(helper.toDeg(xs[p], 2)) for p in parts]
    ws = []
    accuracies = []
    rmse_trs = []
    rmse_tes = []
    for p in parts:
        w, accuracy, rmse_tr, rmse_te = train_model_ridge_cross(xpow[p], ys[p], lambdas=lambdas)
        ws.append(w)
        accuracies.append(accuracy)
        rmse_trs.append(rmse_tr)
        rmse_tes.append(rmse_te)
        
    return ws, accuracies, rmse_trs, rmse_tes

In [56]:
train_all_models(x, y)

([array([ -1.11367490e-01,   3.14853132e-01,  -3.40984572e-01,
           2.69156225e-01,   1.67071379e-01,  -1.91768188e-02,
           1.61166493e-01,  -2.43514710e-01,  -4.23768444e-02,
           8.49293596e-02,   5.92607580e-04,   1.10469641e-06,
           9.51867299e-02,   2.32016087e-03,   2.31044905e-05,
          -1.78091482e-01,  -2.27780775e-05,   4.98413753e-02,
           2.46555294e-02,   1.51295264e-01,  -2.42095410e-01,
          -4.83883735e-02,   4.06545977e-03,   2.73583558e-02,
           1.00560003e-01,   4.35113631e-03,   9.51756590e-03,
          -5.68093164e-06,   1.79023418e-08,  -1.31182728e-01,
          -4.79589745e-05,   1.73227029e-07,   6.30777652e-02,
           3.21068961e-07,  -6.87051800e-03]),
  array([  1.84502362e-02,   1.93284716e-01,  -1.51542269e-01,
           8.08032532e-02,  -1.16188278e-02,   1.38650274e-02,
           1.58460686e-02,   7.45287543e-02,  -1.03290014e-01,
           3.57154944e-02,   1.42961570e-01,   5.97649756e-05,
        

In [54]:
print(accuracy)

0.833208226993


In [25]:
denom = sum(data[p][0].shape[0] for p in part)

In [26]:
num = sum(data[p][0].shape[0] * accuracies[p] for p in part)

In [27]:
print(num/denom)

0.790384270975


In [30]:
print(rmse_tes)

[[0.35087672935786351, 0.35530856533602284], [0.40798808345342413, 0.40384106538188791], [0.39689234540604512, 0.38849535529802492], [0.43274708995280037, 0.42912059642360056]]
